## Знакомство с линейной регрессией 📈
Линейная регрессия — один из самых простых и широко используемых методов, используемых для анализа данных и прогнозирования. Она позволяет моделировать линейные зависимости между переменными, представляя её в виде прямой линии. 
* Основная цель линейной регрессии — предсказать значение зависимой переменной на основе независимых, называемых факторами или более модно - фичами. 

Однако, линейная регрессия не так проста и бывает разных видов:
- 📏 _"Простая" линейная регрессия_ - регрессия с одним признаком
- 🧮 _"Многомерная" линейная регрессия_ - регрессия с несколькими признаками
- 🌀 _"Полиноминальная" регрессиия_ - регрессия с несколькими признаками + степени этих признаков (полином)
- 📊 _"Регрессия Пуассона"_ - модель линейной регресси, которая лучше работает когда целевая переменная — счётные данные (число заказов, покупок, покупателей, ...)

Линейная регрессия является отличной отправной точкой (бейзлайном) перед построением или сравнением с более сложными моделями. 

В этом ноутбуке мы познакомимся с моделью линейной регрессии с одним признаком.

In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

## Данные 📊
При изучении чего-либо я сторонник использовать подход от простого к сложному, поэтому данные будут очень простыми - всего пара столбцов и немного строк. Будем работать с данными о рекламе и продажах. У нас есть 200 рынков, на каждом из них мы инвестировали в разную рекламу какую-то сумму и получили продажи `Sales`

- `TV` - доллары, потраченные на ТВ рекламу для одного продукта на данном рынке (в тысячах)
- `Radio` - доллары, потраченные на радио рекламу
- `Newspaper` - доллары, потраченные на рекламу в газетах
- `Area` - в какой зоне происходила реклама (сельская, пригород, город)
- `Sales` - продажи одного продукта на данном рынке

Предсказывать мы будем продажи - это будет наша целевая переменная, остальные - объясняющие.

In [2]:
# загрузим данные
data = pd.read_csv('../data/regression/ads_data.csv')
data.head()

,TV,Radio,Newspaper,Sales,Area
0,230100.0,37800.0,69200.0,22100.0,suburban
1,44500.0,39300.0,45100.0,10400.0,urban
2,17200.0,45900.0,69300.0,9300.0,rural
3,151500.0,41300.0,58500.0,18500.0,rural
4,180800.0,10800.0,58400.0,12900.0,suburban


- Например, в первой строчке мы получили продажи в размере 22.1k $, вложив:
    - 230.1k $  в канал _TV_
    - 37.8k $ в _Radio_
    - 69.2k $ в _Newspaper_ в пригороде

## Регрессия с одним признаком 📉
Простая линейная регрессия - это подход к прогнозированию *количественной* переменной с использованием *одного признака*:

$y = w_0 + w_1x_1$

- $y$ - таргет
- $x$ - признак
- $w_0$ - свободный член (bias или смещение)
- $w_1$ - вес/коэффициент при признаке $x_1$

Вместе $w_0$ и $w_1$ называются *коэффициентами модели* или *весами*. Чтобы создать модель, мы должны "выучить" значения этих коэффициентов. И как только мы "выучим" эти коэффициенты, мы сможем использовать модель для прогнозирования. В конспекте я упоминал, что найти веса модели можно 2-мя способами:
- 🧠 _Аналитическое решение_ - просто используем формулу
- ⚙️ _Численное решение_ - используем градиентный спуск

Попробуем найти решение для нашей простой модели, используя 2 выше упомянутых метода, используя лишь 1 признак - `TV`. Это можно сделать, используя все признаки, но для простоты я рекомендую сфокусироваться на 1 признаке.

In [3]:
# отберем признак "TV"
columns = ['TV', 'Sales']
data_single = data[columns]
data_single.head()

,TV,Sales
0,230100.0,22100.0
1,44500.0,10400.0
2,17200.0,9300.0
3,151500.0,18500.0
4,180800.0,12900.0


- Теперь у нас есть признак `TV`, используя который мы попытаемся предсказать целевую переменную (продажи)

In [4]:
from sklearn.model_selection import train_test_split 

# также не забудем разделить выборку на train/test в пропроции 80/20
X_train, X_test, y_train, y_test = train_test_split(
    data_single['TV'], data_single['Sales'], test_size=0.2, random_state=42, shuffle=True
)

# проверим размерности полученных данных
print('Кол-во наблюдений (Train) -> ', X_train.shape[0])
print('Кол-во наблюдений (Test) -> ', X_test.shape[0])

Кол-во наблюдений (Train) ->  160
Кол-во наблюдений (Test) ->  40


- Получили 160 строчек/наблюдений в train и 40 в test: это соответствует пропорции нашего разбиения.

## Аналитическое решение 🧠
Давай посмотрим как идёт оптимизация на простом примере. Удалим константу и рассмотрим модель с одним регрессором. 

__Модель:__ 

$$
y_i = w \cdot x_i
$$

__Функция потерь:__

$$
MSE = \frac{1}{n} \sum_{i=1}^n (y_i - \hat y_i)^2 =  \frac{1}{n} \sum_{i=1}^n (y_i - w \cdot x_i)^2 \to \min_{w}
$$


Попробуем найти оптимальное $w$, взяв производную и приравняв её к нулю. 

$$
\frac{\partial MSE}{\partial w} = - \frac{2}{n} \sum_{i=1}^n (y_i - w \cdot x_i) x_i = 0
$$

__Решение:__

$$
{w} = \frac{ \sum x_i y_i }{ \sum x^2_i}

In [5]:
# решим аналитически, предварительно трансформировав данные в вектора
np.sum(X_train.values * y_train.values) / np.sum(X_train.values**2)

np.float64(0.08262850261766856)

## Пару слов про библиотеки 📚
Для обучения большинства ML-моделей достаточно использования [sklearn](https://scikit-learn.org/stable/index.html). Однако, есть еще одна удобная библиотека которая также часто используется - [statsmodels](https://www.statsmodels.org/stable/index.html).
- _Sklearn_ больше направлен на "прикладной" ML - обучение моделей, их валидация и усовершенствование
- _Statsmodels_ больше про статистику и интерпретацию моделей. В нем зашит автоматический расчет доверительных интервалов, значимости коэффициентов и много другое. Основной акцент сделан на интерпретации модели с статистичейской точки зрения

И формат с точки зрения кода немного отличается, поэтому _statsmodels_ требует небольшого привыкания. Так как _sklearn_ очень часто используется при обучении ML-моделей, то мы именно на нем и сфокусируемся! _Statsmodels_ будет рассмотрен в дополнительных уроках.

In [6]:
# теперь решим с использованием модели из модуля sklearn
from sklearn.linear_model import LinearRegression

# объявляем модель. Так как мы игнорировали константу, то указываем в параметрах "fit_intercept=False" (иначе веса не сойдутся)
regression_model = LinearRegression(fit_intercept=False)

# обучаем модель на train (тренировочная выборка) 
regression_model.fit(X_train.to_frame(), y_train)

# смотри на выученный вес 
regression_model.coef_

array([0.0826285])

- Полученные веса совпадают, значит мы все сделали верно!
- Мы передали DataFrame в методе `fit()`, так как [_sklearn_](https://scikit-learn.org/stable/index.html) ожидает двумерную матрицу признаков формата *(n_samples, n_features)*, даже если у тебя всего один признак! Это нормально, так как ML алгоритмы обучаются быстрее и эффективнее при использовании векторов и матриц!

In [7]:
# можно передавать не DataFrame, а уже преобразованную матрицу (n_samples x n_features) в метод fit:
X_train_matrix = X_train.values.reshape(-1, 1)
X_test_matrix = X_test.values.reshape(-1, 1)
print('Размерность матрицы признаков X (Train) -> ', X_train_matrix.shape)
print('Размерность матрицы признаков X (Test) -> ', X_test_matrix.shape)

Размерность матрицы признаков X (Train) ->  (160, 1)
Размерность матрицы признаков X (Test) ->  (40, 1)


- -1 означает "подбери число строк сам так, чтобы размерность осталась совместимой с исходным массивом"

In [8]:
# обучаем модель
regression_model.fit(X_train_matrix, y_train)

# смотри на выученный вес 
regression_model.coef_

array([0.0826285])

- Результат также совпадает 🎉 
- Теперь проверим качество модели - как сильно она ошибается? 🤔

In [9]:
from sklearn.metrics import mean_squared_error

# измеряем качество на обучении (train) и тестовых данных (test)
preds_train = regression_model.predict(X_train_matrix)
preds_test = regression_model.predict(X_test_matrix)

# измеряем ошибки
mse_train = mean_squared_error(y_train, preds_train)
mse_test = mean_squared_error(y_test, preds_test)

print('MSE (Train) -> ', mse_train)
print('MSE (Test) -> ', mse_test)

MSE (Train) ->  22736345.014394596
MSE (Test) ->  24308454.737048913


- Получили ошибку в 22 **квадратных миллионов долларов** на обучающей выборке - _train_ и 24 квадратных миллионов долларов на тестовой выборке _test_. Анализировать квадратные миллионы долларов не очень понятно, особенно если необходимо предоставить результаты бизнесу или твоему начальнику, лиду, ... 
- Для избавления от квадратов используют квадрат из среднеквадратичной ошибки - ["Root Mean Squared Error"](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html).

In [10]:
from sklearn.metrics import root_mean_squared_error as rmse

# измеряем ошибки (без квадратов, используя RMSE)
rmse_train = rmse(y_train, preds_train)
rmse_test = rmse(y_test, preds_test)

print('RMSE (Train) -> ', rmse_train)
print('RMSE (Test) -> ', rmse_test)

RMSE (Train) ->  4768.264360791523
RMSE (Test) ->  4930.360507817752


- Используя лишь 1 признак наша модель ошибается на _4768$_ на обучающей выборке, и на _4930$_ на тестовой. Много это или мало мы должны решить на основании бизнес требований. Однако пока это не является нашей целью.  

## Интерпретация коэффициентов модели 🔍
Отлично, ты обучил модель и теперь знаешь чему равен коэффициент `w`. С математической точки зрения твоя модель имеет следующий вид:

$$
\hat y = 0.0826285 \cdot x
$$

Известно, что одним из преимуществ линейных моделей является их интерпретируемость. Как интерпретировать данную модель? 🤔 

Ну, так как модель линейная, то и признаки интерпретируются соответствующим образом:

> Коэффициент при признаке `w` показывает, насколько изменится целевая переменная `y` при изменении признака `x` на единицу.

- Если `w > 0`: увеличение `x` приводит к росту `y` (прямая зависимость)
- Если `w < 0`: увеличение `x` приводит к уменьшению `y` (обратная зависимость)
- Если `w = 0`: зависимости нет (признак никак не влияет на таргет)

В нашем случае, коэффициент равен 0.0826285. Это значит, что при увеличении `TV` на единицу, наш таргет увеличивается на 0.08. С точки зрения бизнеса, это звучало бы вот так:

> C каждого проинвестированного доллара в рекламу на TV мы получаем 0.08 долларов продаж.

## Задание 💪
- Как думаешь, следует ли продолжать инвестировать в TV канал, используя модель выше?

In [11]:
# твои мысли 🧠

## Градиентный спуск ⛰️
Аналитическое решение - это неплохо. Однако его не всегда можно применить + у него есть недостатки, например, очень медленный при большом числе признаков. В таких случаях "спасает" метод градиентного спуска:

1. Инициализируем случайно значение w_0
2. Находим производную функции потерь $\frac{\partial L}{\partial w}$
3. До тех пор, пока $|w_{t + 1} - w_t| > \varepsilon$ делаем шаги

$$
w_{t+1} = w_t - \eta_t \cdot \frac{\partial L}{\partial w}
$$

В качестве скорости обучение, $\eta_t$, можно взять какую-нибудь малую константу. Например, $0.001$, либо подобрать это значение более умным образом - делить на номер шага/итерации.

In [12]:
# определим функцию вычисления градиента MSE (просто производная от MSE)
def calculate_gradient_mse(
    x: np.ndarray, y: np.ndarray, y_pred: np.ndarray
) -> float:
    n = y.shape[0]
    return (-2 / n) * np.sum(x * (y - y_pred)) 

In [13]:
# данные в виде векторов
x = X_train.values  # вектор размерности (160, )
y = y_train.values  # вектор размерности (160, )

# основные параметры алгоритма
weights_history = [0.05]  # здесь будем хранить историю обновления весов модели
mse_history = []          # здесь будем хранить историю изменения ошибки
eta = 1e-6                # скорость обучения
eps = 1e-7                # разница между весами после которой останавливаем алгоритм
i = 0                     # индекс начала цикла
weights_diff = 100        # изначальная разница между новыми весами и старыми
show_loss = True          # будет печатать значение ошибки

# логика алгоритма (градиентный спуск)
while weights_diff > eps:

    # прогнозируем
    y_pred = weights_history[-1] * x
    
    # вычисляем MSE
    mse = np.mean((y - y_pred)**2)

    # вычисляем градиент ошибки
    mse_grad = calculate_gradient_mse(x=x, y=y, y_pred=y_pred)

    # вычисляем новые веса (формула градиентного спуска)
    w_new = weights_history[-1] - eta * mse_grad

    # вычисляем раницу между весами (чтобы понимать когда останавливать алгоритм градиентного спуска)
    weights_diff = np.abs(w_new - weights_history[-1])

    # сохраянем веса модели и ошибку
    weights_history.append(w_new)
    mse_history.append(mse)
    i += 1

    if show_loss:
        if i % 10 == 0:
            print('MSE -> ', mse)

# выведем информацию о числе итераций
print('\nЧисло итераций (Градиентный Спуск): ', len(weights_history))
print('Оптимальный вес: ', weights_history[-1])

MSE ->  2.493142998739777e+93
MSE ->  6.909685854142796e+188
MSE ->  1.9150028148034174e+284
MSE ->  inf
MSE ->  inf
MSE ->  inf

Число итераций (Градиентный Спуск):  66
Оптимальный вес:  nan


- Получили бесконечность, хмм 🤔. Что могло пойти не так? Скорее всего что-то не так на этапе обновления весов. Давай взглянем как изменялись веса и ошибка

In [14]:
# как "взрывались" веса модели 💥
k = 5
print('Взрыв весов модели:')
print(weights_history[:k])

# как росла ошибка модели
print('\nРост ошибки модели:')
print(mse_history[:k])

Взрыв весов модели:
[0.05, np.float64(1930.8622187499998), np.float64(-114253168.9885492), np.float64(6760889077522.698), np.float64(-4.000731138589978e+17)]

Рост ошибки модели:
[np.float64(54236100.78125), np.float64(1.1030070745950608e+17), np.float64(3.862330283161564e+26), np.float64(1.352447827660439e+36), np.float64(4.7357812316511514e+45)]


Wow, наши веса очень сильно растут от итерации к итерации, то же самое происходит и с ошибкой модели. На каждой итерации веса становятся все больше и больше, это приводит к очень большим предсказаниям и большой ошибке, которую мы еще возводим в квадрат. Такие значения просто не помещаются в тип данных `float`, что приводит к появлению бесконечности. Получается, что вместо минимизации алгоритм совершает максимизацию. А это нам не нужно! Проблема связана с разным масштабом признаков `TV` и `Sales` ☝️

In [15]:
# масштаб перменной X
print('Диапазон изменения переменной "TV":')
print(f'[{X_train.min()} - {X_train.max()}]')

# масштаб перменной y
print('\nДиапазон изменения переменной "Sales":')
print(f'[{y_train.min()} - {y_train.max()}]')

Диапазон изменения переменной "TV":
[700.0 - 296400.0]

Диапазон изменения переменной "Sales":
[1600.0 - 27000.0]


## 📏 Проблема разного масштаба признаков
- Максимум признака `TV` ≈ 296 400, если округлить, то получим ≈ 300 000
- Максимум признака `Sales` ≈ 27 000 → округлённо ≈ 30 000

То есть `TV` примерно в 10 раз больше, чем `Sales` — разница в один порядок (10¹). Этого уже достаточно чтобы алгоритм перестал сходиться, и вот почему: если взглянуть на формулу градиента линейной регресии:

$$
\frac{\partial \text{MSE}}{\partial w} = -\frac{2}{n} \sum_{i=1}^{n} (y_i - w x_i) \cdot x_i
$$

то заметим, что здесь два аргумента `x_i`:
- Первый — напрямую в `w * x_i`
- Второй — умножается на остаток `(y - w * x_i)`

Теперь если например `x = 100 000`, а `y = 20 000`, тогда:
- Умножение `w * x_i = большое число`, а при вычитании `(y - w * x_i) = большой остаток`
- Умножаем полученный остаток на ещё одно `x_i` → градиент становится очень большим ("взрывается")
- Если градиент слишком большой, то шаг обновления тоже становится большим:
$$
w_{\text{new}} = w_{\text{old}} - \eta \cdot \text{grad}
$$
- Даже при малом `η` шаг может быть нестабильным → "взрыв" весов → `inf` или `nan` → алгоритм не обучается (расходится)

> Масштабировние признаков приводит к тому, что произведение `W*X` больше не "взрывается" и градиенты становятся "стабильными". 

Вот еще небольшая оценка как входное значение  `x_i` влияет на полученный градиент:

| x_i   | y_i - w * x_i | Градиент              |
|----------|------------------|------------------------|
| 100 000  | -80000 | -8000000000 |
| 1.0      | -0.8     | -0.8              |

Теперь сделаем все тоже самое только предварительно отмасштабировав выборку. Методы масштабирования позволяют просто изменить масштаб значений признакак. Например, сделать так чтобы все значения изменялись в диапазоне от $[0, 1]$ или $[-1, 1]$. Существуют разные методы масштабирования, но пока мы рассмотри тольок один из них - [StandardSclaer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

Может быть многим знаком из статистики - "Z-преобразование" 

In [16]:
from sklearn.preprocessing import StandardScaler

# произведем масштабирование фичей и таргета
scaler_features = StandardScaler()
scaler_target = StandardScaler()

X_train_scaled = scaler_features.fit_transform(X_train_matrix)
X_test_scaled = scaler_features.transform(X_test_matrix)

y_train_scaled = scaler_target.fit_transform(y_train.to_frame())
y_test_scaled = scaler_target.transform(y_test.to_frame())

- Метод `.fit_transform()` в `StandardScaler` ожидает матрицу на входе, поэтому я преобразовал `X_train` в `DataFrame`. Можно было преобразовать и в матрицу, используя NumPy

In [17]:
print('Отмасштабированные значения признака "TV":')
print(X_train_scaled[:k].reshape(-1))

print('\nОтмасштабированные значения таргета "Sales":')
print(y_train_scaled[:k].reshape(-1))

Отмасштабированные значения признака "TV":
[-0.40424839  0.32060772 -1.27051084 -1.04235941  0.8791034 ]

Отмасштабированные значения таргета "Sales":
[-0.60870673 -0.25526411 -0.78542804 -0.86397084 -0.49089252]


Можно заметить, что появились отрицательные значения. Это нормально, так как `StandardScaler` масштабирует данные по формуле:

$$
x_{\text{scaled}} = \frac{x - \text{mean}(x)}{\text{std}(x)}
$$

- Значения меньше среднего становятся отрицательными, а больше среднего - положительными
- Также я отмасштабировал не только матрицу признаков `X`, но и таргет `y` ☝️

Если не масштабировать `y`, то получится сильный "перекос" градиента функции, так как наш `X` изменяется в пределах от [-1, +1], а целевая переменная в десятках тысяч. Этого уже достаточно, чтобы снова получить проблемы при использовании градиентного спуска 😬. Z-преобразование позволяет осуществить так называемо "центрирование" выборки - то есть сместить данные так, чтобы среднее значение признака стало равно нулю, а стандартное отклонение — единице. Это делает выборку **симметричной относительно нуля**

* _Основная цель сделать выборку симметричной именно относительно нуля!_

In [18]:
# вычислим средние значения для оригинальных и масштабированных данных
X_train_mean = X_train.mean()
y_train_mean = y_train.mean()

X_train_scaled_mean = X_train_scaled.mean()
y_train_scaled_mean = y_train_scaled.mean()

In [21]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# визуализируем оригинальную и центрированную выборки
fig = go.Figure()

# оригинальные данные
fig.add_scatter(
    x=X_train, y=y_train, mode='markers',
    name='Выборка Оригинал (Train)'
)

# добавим точу центра выборки
fig.add_scatter(
    x=[X_train_mean], y=[y_train_mean], mode='markers',
    marker=dict(size=12, color='blue', symbol='x'),
    name='Центр Выборки Оригинал (Train)'
)

# масштабированные/центрированные данные
fig.add_scatter(
    x=X_train_scaled.flatten(),
    y=y_train_scaled.flatten(), mode='markers',
    name='Выборка Центрированная (Train)',
    marker=dict(color='red')
)

# добавим точу центра выборки
fig.add_scatter(
    x=[X_train_scaled_mean], y=[y_train_scaled_mean], mode='markers',
    marker=dict(size=12, color='red', symbol='x'),
    name='Центр Выборки Центрированная (Train)'
)


fig.update_layout(
    title='Зависимость между "Инвестициями в рекламу на TV" и "Продажами" (Train)',
    xaxis_title='TV Spent $',
    yaxis_title='Sales',
    height=900,
    width=1400,
)

fig.show()

Хорошо видно, что при центрировании данных облако точек смещается в центр координат — там, где признаки равны нулю, выборка симметрична относительно нуля! В случае нецентрированных данных эта точка очень сильно смещена. Причём, чем больше выбросов в выборке, тем сильнее проявляется это смещение.

In [22]:
# запустим алгоритм снова
x = X_train_scaled.reshape(-1)  # вектор размерности (160, )
y = y_train_scaled.reshape(-1)  # вектор размерности (160, )

# основные параметры алгоритма
weights_history = [0.05]
mse_history = []
eta = 1e-2
eps = 1e-7
i = 0
weights_diff = 100
show_loss = True

# логика алгоритма
while weights_diff > eps:

    # прогнозируем
    y_pred = weights_history[-1] * x
    
    # вычисляем MSE
    mse = np.mean((y - y_pred)**2)

    # вычисляем градиент ошибки
    mse_grad = calculate_gradient_mse(x=x, y=y, y_pred=y_pred)

    # вычисляем новые веса (формула градиентного спуска)
    w_new = weights_history[-1] - eta * mse_grad

    # вычисляем раницу между весами (чтобы понимать когда останавливать алгоритм градиентного спуска)
    weights_diff = np.abs(w_new - weights_history[-1])

    # сохраянем веса модели и ошибку
    weights_history.append(w_new)
    mse_history.append(mse)
    i += 1

    if show_loss:
        if i % 10 == 0:
            print('MSE -> ', mse)

# выведем информацию о числе итераций
print('\nЧисло итераций (Градиентный Спуск): ', len(weights_history))
print('Оптимальный вес: ', weights_history[-1])

MSE ->  0.7680648451209262
MSE ->  0.6486591162369593
MSE ->  0.5689428997607954
MSE ->  0.5157237181631535
MSE ->  0.48019416827828587
MSE ->  0.45647435754227794
MSE ->  0.44063882280639693
MSE ->  0.430066893579718
MSE ->  0.4230089893511565
MSE ->  0.4182970762242849
MSE ->  0.41515136545856796
MSE ->  0.4130512638745395
MSE ->  0.4116492193155466
MSE ->  0.41071320319120697
MSE ->  0.41008831136490664
MSE ->  0.4096711286001839
MSE ->  0.40939261406077615
MSE ->  0.4092066755340179
MSE ->  0.4090825414912976
MSE ->  0.40899966861481135
MSE ->  0.40894434202182695
MSE ->  0.4089074055473005
MSE ->  0.4088827464624581
MSE ->  0.40886628386084106
MSE ->  0.4088552932967658
MSE ->  0.40884795590857503
MSE ->  0.40884305740972693
MSE ->  0.4088397871328464
MSE ->  0.4088376038699312
MSE ->  0.40883614630620446
MSE ->  0.4088351732250411
MSE ->  0.40883452358829936
MSE ->  0.4088340898856318
MSE ->  0.4088338003422735
MSE ->  0.4088336070408194
MSE ->  0.40883347799122766
MSE ->  0.4088

- Алгоритм сходится, количество итераций алгоритма зависит от параметров: `eta` и `eps`
- Полученный вес отличается от того что мы видели раньше, но это вполне ожидаемо так как теперь мы обучали модель на отмасштабированных данных!

In [23]:
# создаём фигуру с двумя подграфиками
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,               # общая ось X для обоих графиков
    vertical_spacing=0.1,            # расстояние между графиками
    subplot_titles=("Изменение весов модели", "Изменение ошибки MSE (Train)")
)

# добавляем график - "Изменения весов модели"
fig.add_trace(
    go.Scatter(
        y=weights_history,              # значения весов по оси Y
        mode='lines',                   # тип графика — линия
        name='Веса модели'
    ),
    row=1, col=1                        # расположение графика - поместить в 1-ю строку, 1-й столбец
)                    

# добавляем график - "Изменения ошибки"
fig.add_trace(
    go.Scatter(
        y=mse_history,                  # значения ошибок по оси Y
        mode='lines',                   # тип графика — линия
        name='MSE'
    ),
    row=2, col=1
)

# общие настройки макета графика
fig.update_layout(
    height=700, width=800,
    title_text="Динамика обучения модели (Градиентный спуск)",
    showlegend=False
)

# добавляем подписи осей
fig.update_xaxes(title_text="Номер итерации", row=2, col=1)
fig.update_yaxes(title_text="Вес", row=1, col=1)
fig.update_yaxes(title_text="Ошибка", row=2, col=1)

# отображаем график
fig.show()

Можно сделать следующие выводы:
- Градиентный спуск успешно сошёлся примерно за 300 итераций.
- Дальнейшие итерации не дают прироста в качестве, но и не вредят — это хороший признак стабильности.
- Выбранная скорость обучения `η` подходит — не вызывает ни колебаний, ни слишком долгого обучения.

In [28]:
# теперь используем вычисленный вес методом градиентного спуска в модели и измеряем качество
weight = weights_history[-1]
model = LinearRegression(fit_intercept=False)

# вручную задаем коэффициенты
model.coef_ = np.array([weight])
model.intercept_ = 0.0  # так как fit_intercept=False

# измеряем качество на обучении (train) и тестовых данных (test)
preds_train = model.predict(X_train_scaled)
preds_test = model.predict(X_test_scaled)

# измеряем ошибки (без квадратов, используя RMSE)
rmse_train = rmse(y_train_scaled, preds_train)
rmse_test = rmse(y_test_scaled, preds_test)

print('RMSE (Train) -> ', rmse_train)
print('RMSE (Test) -> ', rmse_test)

RMSE (Train) ->  0.6394006715820137
RMSE (Test) ->  0.6272577065406721


Модель ошибается на 0.62$ на тестовой выборке. Вау, замечательный результат! 🎉 Однако не спеши радоваться, ты получил результат на отмасштабированных данных, <u>и это не доллары, а еденицы стандартного отклонения таргета!</u> - так как мы масштабировали его! Если хочешь узнать ошибку модели на реальных долларах, то тебе необходимо инвертировать результат масштабирования!

In [29]:
# инвертируем предсказания
preds_train = scaler_target.inverse_transform(preds_train.reshape(-1, 1))
preds_test = scaler_target.inverse_transform(preds_test.reshape(-1, 1))

rmse_train = rmse(y_train, preds_train)
rmse_test = rmse(y_test, preds_test)

print('RMSE (Train) -> ', rmse_train)
print('RMSE (Test) -> ', rmse_test)

RMSE (Train) ->  3256.3170256607445
RMSE (Test) ->  3194.475764674437


Вот теперь ты смотришь на реальные значения ошибки модели в долларах. 
- На обучении модель в среднем ошибается на 3256$, а на тестовых данных на 3194$.
- Методы которые мы использовали для получения оптимальных весов модели принесли разные результаты:
    - ✍️ _Аналитическое решение:_ RMSE (Test) -> 4930$
    - 📉 _Градиентный спуск:_ RMSE (Test) -> 3194$

Можно предположить, что масштабирование выборки позволило лучше сойтись методу градиентного спуска, чем аналитическое решение. Градиентный спуск дал на ~35.2% меньше RMSE, чем аналитическое решение — по тестовой выборке. Это круто! Вот возможные причины:
- Градиентный спуск — не дошёл до "аналитического" минимума, а остановился раньше. 
    - Например, из-за `eps` или малой скорости обучения. Это могло случайно защитить от переобучения — своего рода "early stopping".
- Аналитическое решение очень точно подогнало модель под тренировочные данные, включая шум, что ухудшило обобщающую способность (overfitting).
- Масштабирование признаков `X` и `y` улучшило численную стабильность градиентного спуска, а аналитическое решение — страдало от численных ошибок - особенно если диапазон признака `TV` большой.

## 🧪 Base-line модели
Base-line (базовая) модель — это простейшая модель, с которой мы сравниваем качество всех последующих, более сложных моделей. Она играет роль отправной точки в процессе построения ML-системы.

Скорее всего у тебя возникает вопрос: "А зачем их использовать?"
- Адекватность модели: позволяет убедиться, что более сложная модель действительно учится чему-то полезному.
- Оценка метрик: новая модель должна действительно работать лучше предыдущей
- Диагностика данных: если даже простая модель работает плохо — возможно у тебя проблемы в данных (много шума, плохое качество)

Окей, а какие base-line модели обычно используют — простые, поэтому они так и называются "base-line". Во список base-line моделей применяемых для регресии:
- Среднее значение
    - $\hat{y} = \bar{y}_{\text{train}}$  
- Медианное значение
    - $\hat{y} = median({y}_{\text{train}})$
- Линейная регрессия без сложного препроцессинга данных и отбора признаков

Если ты потратил кучу времени и сил на обучение модели, а затем оказывается, что она работает чуть лучше или даже хуже, чем base-line, то значит ты где-то "накосячил". В таком случае тебе необходимо будет проверить все этапы с самого начала и найти ошибку. Если модель слишком сложная, то скорее всего ты переобучаешься.

## 🐣 Sklearn DummyRegressor
В sklearn уже работает все из коробки. Для прогнозирования средним или медианой, используй [DummyRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html). Данный регрессор поддерживает множество различный вариантов построения base-line моделей: смотри параметр `strategy`

In [30]:
from sklearn.dummy import DummyRegressor

# определим 2 base-line модели
mean_model = DummyRegressor(strategy='mean')
median_model = DummyRegressor(strategy='median')

# обучим модели на тренировочной выборке
mean_model.fit(X_train_matrix, y_train)
median_model.fit(X_train_matrix, y_train)

# провалидируем модели на тестовой выборке
mean_preds_train = mean_model.predict(X_train_matrix)
median_preds_train = median_model.predict(X_train_matrix)
mean_preds_test = mean_model.predict(X_test_matrix)
median_preds_test = median_model.predict(X_test_matrix)

# измерим качество моделей
rmse_mean_train = rmse(y_train, mean_preds_train)
rmse_median_train = rmse(y_train, median_preds_train)
rmse_mean_test = rmse(y_test, mean_preds_test)
rmse_median_test = rmse(y_test, median_preds_test)

print('RMSE | Mean Model | Train -> ', rmse_mean_train)
print('RMSE | Mean Model | Test -> ', rmse_mean_test)
print('\nRMSE | Median Model | Train -> ', rmse_median_train)
print('RMSE | Median Model | Test -> ', rmse_median_test)

RMSE | Mean Model | Train ->  5092.76447521383
RMSE | Mean Model | Test ->  5631.496248777939

RMSE | Median Model | Train ->  5171.677677504661
RMSE | Median Model | Test ->  5641.475870727447


- Так как в данных не особо много выбросов, то модели практически не отличаются друг от друга по качеству.

Основная цель построения base-line моделей это понимать примерный рост качества последующих моделей или улучшений, то давай сравним, насколько модель линейной регрессии превосходит простую модель, которая всегда предсказывает среднее значение таргета. Вот так я посчитал процентное улучшение _модели B_ относительно _модели A_:

$$
\text{Улучшение (\%)} = \left( \frac{\text{MSE}_\text{A} - \text{MSE}_\text{B}}{\text{MSE}_\text{A}} \right) \cdot 100
$$

Разумеется, вместо _MSE_ можно использовать любу метрику: _RMSE_, _MAE_, ...

In [34]:
# закодим метрику
def calculate_percentage_improvement(metric_a, metric_b):
    return (metric_a - metric_b) / metric_a * 100

# рассчитаем процентное улучшение модели B относительно модели A
imporvement_pct = calculate_percentage_improvement(
    metric_a=rmse_mean_test, metric_b=rmse_test
)

print(f'Процентное улучшение модели B относительно модели A: {imporvement_pct:.2f}%')

Процентное улучшение модели B относительно модели A: 43.27%


- Если сравнить построенную модель с base-line, то заметим, что модель дает на 43% лучшее качество чем base-line - круто! 💪

## Визуализация моделей 🎨
Теперь давай посмотрим как выглядит зависимость между признаком `TV` и `Sales` и визуально сравним наши модели. Напоминаю, я показал 2 способа обучения: аналитическое и численное. Возникает вопрос: "А какую выборку использовать чтобы оценить как хорошо модель обучилась?". Ну, так как мы обучаем модель на тренировочной выборке, то именно на ней и необходимо посмотреть как модель подстроилась под данные.

In [35]:
# аналитическое решение
preds_train_analytical = regression_model.predict(X_train.to_frame())
preds_test_analytical = regression_model.predict(X_test.to_frame())

# градиентный спуск
preds_train_gradient = model.predict(X_train_scaled)
preds_test_gradient = model.predict(X_test_scaled)

preds_train_gradient = scaler_target.inverse_transform(preds_train_gradient.reshape(-1, 1)).reshape(-1)
preds_test_gradient = scaler_target.inverse_transform(preds_test_gradient.reshape(-1, 1)).reshape(-1)

In [36]:
# визуализируем
fig = go.Figure()

fig.add_scatter(
    x=X_train, y=y_train, mode='markers',
    name='Выборка (Train)'
)

fig.add_scatter(
    x=X_train,
    y=preds_train_analytical,
    mode='lines',
    name='Аналитическое решение',
    line=dict(color='blue')
)

fig.add_scatter(
    x=X_train,
    y=preds_train_gradient,
    mode='lines',
    name='Градиентный спуск',
    line=dict(color='red')
)

fig.add_scatter(
    x=X_train,
    y=mean_preds_train,
    mode='lines',
    name='Base-line (Mean)',
    line=dict(color='green')
)

fig.update_layout(
    title='Зависимость между "Инвестициями в рекламу на TV" и "Продажами" (Train Data)',
    xaxis_title='TV Spent $',
    yaxis_title='Sales',
    height=900,
    width=1200,
)

fig.show()

Обе модели за исключением base-line захватывают общую тенденцию: как аналитическое решение, так и градиентный спуск улавливают положительную зависимость между инвестициями в рекламу на ТВ и продажами — чем больше затраты, тем выше продажи.
- _Аналитическое решение_ имеет более крутой наклон, что означает большую чувствительность модели к изменениям в инвестициях. Однако это может привести к переоценке продаж при высоких значениях бюджета.
- _Градиентный спуск_ даёт более пологую линию, которая лучше проходит по центру облака точек, особенно в области высоких значений TV. Это может объяснять, почему у градиентного спуска _RMSE_ оказался ниже — он менее чувствителен к выбросам.

Таким образом, несмотря на то, что _аналитическое решение_ является точным решением по формуле, _градиентный спуск_ с подобранными гиперпараметрами может дать более устойчивую модель при наличии выбросов и шума в данных.

## 📊 Визуализация качества на тестовой выборке
Обучающая выборка позволяет понять, насколько хорошо модель «подстраивается» под известные данные: улавливает ли она основные тренды, есть ли переобучение и т.д. Но что насчёт тестовой выборки? Как ты уже знаешь - тестовые данные используются для оценки обобщающей способности модели: насколько хорошо она справляется с новыми, не виденными ранее данными. Для визуализации качества на тестовой выборке часто применяются вот такие подходы:
- **Scatterplot**
    - Позволяет оценить, насколько сильно предсказания отклоняются от реальных значений. В идеале тестовые точки должны быть как можно ближе к регрессионной прямой - предсказанным значениям
- **Distribution Analysis**
    - Сравниваем распределение предсказанных значений с распределением реальных значений. Хорошее совпадение — признак корректной генерализации модели.

In [37]:
# Scatter plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=X_test,
    y=y_test,
    mode='markers',
    name='Фактические (Test)',
    marker=dict(color='blue', opacity=0.6),
))

fig.add_trace(go.Scatter(
    x=X_test,
    y=preds_test_gradient,
    mode='lines+markers',        # можно использовать только линию (lines) или только точки (markers)
    name='Предсказания (Test)',
    marker=dict(color='red', opacity=0.6),
))

fig.update_layout(
    title='Сравнение фактических и предсказанных значений (Test Data)',
    xaxis_title='TV Spent $',
    yaxis_title='Sales',
    height=800,
    width=1100,
)

fig.show()

In [39]:
from scipy.stats import gaussian_kde

# Distribution plot
# считаем оценку плотности для фактических и предсказанных значений
kde_test = gaussian_kde(y_test)
kde_pred = gaussian_kde(preds_test_gradient)

# определим диапазон значений по которым бдуем строить плотности
min_value = min(y_test.min(), preds_test_gradient.min())
max_value = max(y_test.max(), preds_test_gradient.max())

# также задаим сколько точек будем строить на графике
n_points = 200

# итоговый диапазон значений
x_vals = np.linspace(min_value, max_value, n_points)

# строим график плотностей
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_vals, y=kde_test(x_vals),
    fill='tozeroy', name='Известные', mode='lines',
    line=dict(color='blue'), opacity=0.6
))

fig.add_trace(go.Scatter(
    x=x_vals, y=kde_pred(x_vals),
    fill='tozeroy', name='Прогноз', mode='lines',
    line=dict(color='red'), opacity=0.6
))

fig.update_layout(
    title='Сравнение плотностей распределений: Известные vs Прогноз (Test Data)',
    xaxis_title='Sales', yaxis_title='Плотность',
    width=1000, height=600
)

fig.show()

Мы построили два графика. Как думаешь, какой из них даёт больше информации о модели? Как понять, глядя только на графики выше, есть ли проблемы с моделью или все хорошо? Смотри, первый график неплох, модель улавливает тренд и относительно хорошо предсказывает данные. И все, больше инсайтов сложно вытащить из данного графика. Все меняется если взглянуть на сравнение распределений. Вот основные выводы:
- Во-первых, предсказанное распределение бимодально, что не соответствует истинному распределению 
- Модель завышает количество малых и больших значений по сравнению с реальными
- Основной пик фактических значений модель предсказывает не с той плотностью и не в том месте.

Все это свидетельствует о недостаточной обученности или переобученности модели. Однако, переобучения здесь быть не может, так как мы уже сравнили метрики на тестовой и тренировочной выборках, сильного расхождения замечено не было. Следовательно, можно сделать только один вывод - модель слишком простая (недообучена) и не может достаточно хорошо предсказывать "Sales", используя только один признак "TV". Необходимо использовать более сложную модель или использовать больше признаков.

## Использование модели 🚀
Отлично, мы обучили нашу модель и оценили ее качество. Теперь возникает вопрос: "А как прогнозировать на новых данных?" Прежде всего необходимо посмотреть на нашу модель:

$$
\hat y = w_0 + w_1 \cdot x
$$

Мы можем легко найти данные параметры в обученной модели, используя аттрибуты `.intercept_` и `.coef_` класса `LinearRegression` 

In [40]:
print('Intercept: b0 -> ', model.intercept_)
print('Веса W -> ', model.coef_)

Intercept: b0 ->  0.0
Веса W ->  [0.76886883]


Уравнение примет вид:
$$
\hat y = 0.76886889 \cdot x
$$

Теперь нас интересует вопрос:
- "Сколько мы получим продаж если инвестируем X долларов в рекламу на TV?" 🤔

Для этого просто необходимо подставить кол-во инвестируемых денег в уравнение выше

In [41]:
# например, я хочу инвестировать 100k долларов в рекламу на TV
tv_investment = 100_000
print(f'Размер инвестиций в рекламу на TV: {tv_investment} $')

Размер инвестиций в рекламу на TV: 100000 $


**Важно** ⚠️
- Помни, что ты обучал модель на отмасштабированных данных! Т.е. ты совершил процедуру преобразования данных перед тем как обучить модель - это называется _data transformation_. Следовательно, твоя модель ожидает аналогичный _transformation_ для любых входных данных! ☝️

In [42]:
# масштабируем (скейлим)
tv_investment_scaled = scaler_features.transform([[tv_investment]])
print(f'Размер инвестиций в рекламу на TV (Scaled): {tv_investment_scaled.item()} $')

# прогнозиурем
sales_forecast_scaled = model.predict(tv_investment_scaled)
sales_forecast = scaler_target.inverse_transform(sales_forecast_scaled.reshape(-1, 1))
print(f'\nПрогноз продаж "Sales" (Scaled): {tv_investment_scaled.item()} $')
print(f'Прогноз продаж "Sales" (Real): {sales_forecast.item()} $')

Размер инвестиций в рекламу на TV (Scaled): -0.594374576598522 $

Прогноз продаж "Sales" (Scaled): -0.594374576598522 $
Прогноз продаж "Sales" (Real): 11772.626586262291 $


Получается, что с 100к \$ инвестиций в рекламу на TV мы получим 11.7k \$ продаж. Сомнительно, но окей .... 👌 Возможно не стоит вливать кучу денег только в рекламу на TV, а распределять бюджет между доступными каналами. Также модель линейной регрессии очень хорошо интерпретируется. Коэффициенты модели показывают, насколько изменятся продажи при увеличении бюджета на ТВ на единицу.

## Интерпретация масштабированных коэффициентов модели 🔍

Это особенно полезно для бизнес-аналитики и объяснения модели заинтересованным сторонам 💵. В нашей модели это значение составляет 0.77 и значит, что "при увеличении масштабированного значения расходов на рекламу на единицу, предсказанное масштабированное значение продаж увеличивается на 0.77"

На практике, это очень неудобно, так как бизнес не понимает что такое "отмасштабированное значение", нужно четко понимать: 
- "Если я инвестирую X, то сколько получу на выходе??" 🤔

В таких случаях просто необходимо пересчитать полученный в модели вес/веса.

## Алгоритм пересчета коэффициентов 🔁
Ниже я приведу пример как пересчитываются веса модели, чтобы мы не теряли их интерпретацию и смогли объяснить "бизнесу". Предварителньо нужно только понимать, что мы масштабировали:
- Только матрицу признаков $X$?
- Матрицу признаков $X$ и таргет $y$?

Это очень важно, так как влияет на формулу перерасчета коэффициентов.

_Пересчет коэффициентов для случая масштабирования таргета и признаков_

Если вспомнить формулу по которой `StandardScaler` масштабирует данные, то получим:

- **Признаки:**
  $$
  x_{\text{scaled}} = \frac{x - \mu_x}{\sigma_x}
  $$

- **Целевая переменная:**
  $$
  y_{\text{scaled}} = \frac{y - \mu_y}{\sigma_y}
  $$

После обучения модель имеет вид:

$$
\hat{y}_{\text{scaled}} = w_{\text{scaled}} \cdot x_{\text{scaled}} + b_{\text{scaled}}
$$

Подставим формулы обратно, чтобы выразить предсказание в исходных значениях:

$$|
y = \sigma_y \cdot \left( w_{\text{scaled}} \cdot \frac{x - \mu_x}{\sigma_x} + b_{\text{scaled}} \right) + \mu_y
$$

Раскроем скобки:

$$
y = \left( \frac{\sigma_y}{\sigma_x} \cdot w_{\text{scaled}} \right) \cdot x + \left( \mu_y - \frac{\sigma_y \cdot \mu_x \cdot w_{\text{scaled}}}{\sigma_x} + \sigma_y \cdot b_{\text{scaled}} \right)
$$

Таким образом, формулы для перерасчета весов примут вид:

- **Коэффициенты:**

  $$
  w_{\text{unscaled}} = \frac{\sigma_y}{\sigma_x} \cdot w_{\text{scaled}}
  $$

- **Свободный член (интерцепт):**

  $$
  b_{\text{unscaled}} = \mu_y - \frac{\sigma_y \cdot \mu_x \cdot w_{\text{scaled}}}{\sigma_x} + \sigma_y \cdot b_{\text{scaled}}
  $$

In [43]:
from typing import Any, Tuple

# определим функцию для пересчета коэффициентов и интерцепта из масштабированных значений в исходный масштаб
def unscale_coefficients(
    scaler_X: Any, scaler_y: Any, coef_scaled: np.ndarray, intercept_scaled: float
) -> Tuple[np.ndarray, float]:
    """
    Пересчитывает коэффициенты и интерцепт линейной регрессии
    из масштабированных значений в исходный масштаб.

    Paramters
    ---------
    scaler_X:
        Scaler, применённый к признакам X_train.
    scaler_y:
        Scaler, применённый к таргету y_train.
    coef_scaled: n.ndarray
        Коэффициенты модели после масштабирования.
    intercept_scaled: float
        Интерцепт модели после масштабирования.
    
    Returns
    -------
    coef_unscaled: np.ndarray
        Коэффициенты в исходном масштабе.
    intercept_unscaled: float
        Интерцепт в исходном масштабе.
    """
    coef_unscaled = coef_scaled * (scaler_y.scale_ / scaler_X.scale_)
    intercept_unscaled = scaler_y.mean_ - np.sum(coef_unscaled * scaler_X.mean_) + intercept_scaled * scaler_y.scale_
    return coef_unscaled, intercept_unscaled

In [44]:
# осуществляем пересчет коэффициентов и интерцепта
coef_unscaled, intercept_unscaled = unscale_coefficients(
    scaler_X=scaler_features,
    scaler_y=scaler_target,
    coef_scaled=model.coef_,
    intercept_scaled=model.intercept_
)

print('Коэффициент модели (Масштабированный):', model.coef_)
print('Коэффициент модели (Пересчитанный):', coef_unscaled)

Коэффициент модели (Масштабированный): [0.76886883]
Коэффициент модели (Пересчитанный): [0.04652944]


- Получается, что реальное значение веса это 0.04, значит если инвестировать 1$ в рекламу, то мы получим 0.04$ прибыли (меньше цента). Ну такое себе 🙃

## Влияние дубликатов 👯‍♂️
Так как мы использовали только один признак, то есть высокая вероятность того, что одинаковое количество инвестиций в рекламу на "TV" могло приводить к разным значениям продаж. Например, инвестировали 200к, но в разных локациях (rural, suburban). 

In [45]:
# првоерим налчие дубликатов
data_single['TV'].value_counts().head(5)

TV
17200.0     2
199800.0    2
240100.0    2
237400.0    2
177000.0    2
Name: count, dtype: int64

- Получается, что в наших данных есть наблюдения которые ведут к одним и тем же значениям таргета. Как думаешь это хорошо или плохо?

In [46]:
data[data['TV'] == 184900.0]

,TV,Radio,Newspaper,Sales,Area
97,184900.0,21000.0,22000.0,15500.0,urban
139,184900.0,43900.0,1700.0,20700.0,rural


- Вот и доказательство, мы вложили 184к $ и получили разное количество продаж. Просто одна реклама проходила в местности _"urban"_, а другая в _"rural"_. А как это повлияет на модель? Давай проверим, ведь мы ее уже обучили:

In [47]:
# небольшой препроцессинг
X_duplicated = data_single[data_single['TV'] == 184900.0].drop(columns='Sales')
X_duplicated_scaled = scaler_features.transform(X_duplicated)

# предсказываем
preds = model.predict(X_duplicated_scaled)
preds = scaler_target.inverse_transform(preds.reshape(-1, 1))
preds

array([[15722.97588244],
       [15722.97588244]])

- В таком случаем модель пытается предсказать что-то среднее или между этими значениями - провести прямую так чтобы ошибка между 2-мя таргетами была минимальна. В нашем случае прогноз не совсем равен среднему 2-х таргетов по этим признакам. Потому что линейная регрессия не подгоняет прямую только под эти две точки. Она оптимизирует всю выборку — то есть она пытается минимизировать сумму квадратов ошибок по всем наблюдениям, а не просто усреднить значения в этих двух 😉

In [48]:
# среднее по таргетам для дубликатов
data[data['TV'] == 184900.0]['Sales'].mean()

np.float64(18100.0)

## Задание 💪
- Для закрепления материала, предлагаю тебе проделать аналогичные шаги, но только для другого признака.
    - Какой получился вес итоговой модели?
    - Лучше ли этот признак в сравнении с признаком `TV`?
    - Оцени качество модели с масштабированием и без него. Есть ли различия в качестве?

In [39]:
# твой код 💻